# Crawler of Stock Price in Taiwan Stock Market

## Acquire The Stock Code

* Originally, since I did not know all of the stock code, I hoped that I could create an array in which there are 10,000 elements, and I could try each element whether I could get the result. However, the time I need to consume is O(10,000) if I use this way. Therefore, I use an alternative.

stockCodeMatrix = []

start = 50
to = 10000
numberOfSecurity = to - start

for i in range(start,to):
    
    if len(str(i)) == 1:
        stockCodeMatrix.append("000" + str(i))
    elif len(str(i)) == 2:
        # "else if" should be written to "elif" in python...
        stockCodeMatrix.append("00" + str(i))
    elif len(str(i)) == 3:
        stockCodeMatrix.append("0" + str(i))
    elif len(str(i)) == 4:
        stockCodeMatrix.append(str(i))
        
for j in range(0,len(stockCodeMatrix)):
    print(stockCodeMatrix[j])

* My alternative is that I download an excel file in which there're all the stock code, and I use each code in this file to check whether I could get the result. The time I need to consume is less than O(2,000) if I use this way, which is more effective.

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

columnArray = ['上市、上櫃股名代碼表', 'Unnamed: 2', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 10', 'Unnamed: 12', 'Unnamed: 14', 'Unnamed: 16', 'Unnamed: 18', 'Unnamed: 20', 'Unnamed: 22', 'Unnamed: 24', 'Unnamed: 26', 'Unnamed: 28', 'Unnamed: 30', 'Unnamed: 32', 'Unnamed: 34', 'Unnamed: 36', 'Unnamed: 38'] 
stockNameArray = ['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 19', 'Unnamed: 21', 'Unnamed: 23', 'Unnamed: 25', 'Unnamed: 27', 'Unnamed: 29', 'Unnamed: 31', 'Unnamed: 33', 'Unnamed: 35', 'Unnamed: 37', 'Unnamed: 39']
stockCodeMatrix = []
stockNameMatrix = []

#print(DFOfStock)    # Check the whole form
pathProg = os.path.abspath('.')
DFOfStock = pd.read_excel(pathProg + '/Supporting Files/StockTable.xlsx')

for r in range(0, len(columnArray)):
    stockCode1 = DFOfStock[columnArray[r]][2:83]
    stockName = DFOfStock[stockNameArray[r]][2:83]
    
    for s in range(0, len(stockCode1.values)):
        try:
            stockCodeMatrix.append(int(stockCode1.values[s]))
            stockNameMatrix.append(str(stockName.values[s]))
        except:
            print("", end = '\t')

print(len(stockCodeMatrix))    # Check whether I have load and store all the codes into the array
print(len(stockNameMatrix))    # Check whether I have load and store all the names into the array

# The website I refered to: https://blog.csdn.net/chenKFKevin/article/details/62049060

																																																																																											1529
1529


## Download HTML Code And Extract The Information We Want To Know

In [29]:
import time

nameArray = []
closePriceArray = []
openPriceArray = []
differencePriceArray = []
startTime = time.time()

for m in range(0, len(stockCodeMatrix)):
    if len(nameArray) < len(stockCodeMatrix):
        url = 'https://tw.stock.yahoo.com/q/q?s='+ str(stockCodeMatrix[m])
        header = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
        'Cookie':'PSTM=1476231684; BIDUPSID=4F526560482E2A5E68D69CC8B0998806; plus_cv=1::m:92e3c68f; BAIDUID=C5A710455602AEA5BEC3D1B13B26321B:FG=1;'
                 ' BDUSS=W5zS3JSeVYwSHZjVm5SdTdjQjlKNC1FLWJqbklvaEptZjVZVkl2bXhMN1o1amhZSVFBQUFBJCQAAAAAAAAAAAEAAACj2nZjanVleWluZ3MAAAAAAAAAAAAAAAAAAAA'
                 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANlZEVjZWRFYT; BD_HOME=1; BD_UPN=12314353; sug=3; sugstore=0; ORIGIN=2; bdime=0;'
                 ' H_PS_645EC=78d5XI4%2Bj6NkSjLKSmkiYdx%2F5jHNa0c4UemYz6WwEpyczIPebiQwaLtzwnXd2gUHv28P; BDRCVFR[feWj1Vr5u3D]=I67x6TjHwwYf0; BD_CK_SAM=1;'
                 ' PSINO=6; H_PS_PSSID=1448_18288_21112_17001_20241_21455_21406_21394_21377_21192_20929; BDSVRTM=0'
    }

        response = requests.get(url,headers=header)

        soup = BeautifulSoup(response.text, 'lxml')
        codeOfAllPrice = soup.find_all('td')
        codeOfTitle = soup.find_all('a')
        endTime = time.time()
        #print("test!!!!!!!!!!!!")
        #print(codeOfAllPrice)
        #print("test!!!!!!!!!!!!")
        #for i in range(1, 20):
        #    print(i)
        #    print(codeOfAllPrice[i].text)
        #    print("Next")
        #break
        
        closePrice = codeOfAllPrice[10].text
        openPrice = codeOfAllPrice[11].text
        highestPrice = codeOfAllPrice[12].text
        lowestPrice = codeOfAllPrice[13].text
        nameOfTarget = stockNameMatrix[m]
    
        try:
            float(closePrice)
            nameArray.append(nameOfTarget)
            closePriceArray.append(closePrice)
            openPriceArray.append(openPrice)
            differencePriceArray.append(str(lowestPrice) + "-" + str(highestPrice))
            
            print("Completition: " + str(int(((m + 1)/len(stockCodeMatrix))*100)) + "%")
            print(len(nameArray))
            
            secondsPassed = endTime-startTime
            restSeconds = ((secondsPassed)/(m+1))*len(stockCodeMatrix)-secondsPassed
            restMinutes = int(round(restSeconds/60, 0))
            residualSeconds = round(restSeconds%60, 3)
            print("Estimated rest of the time:", restMinutes, "minutes", residualSeconds ,"seconds")
            
        except ValueError:
            print("Completition: " + str(int(((m + 1)/len(stockCodeMatrix))*100)) + "%")
            
    elif len(nameArray) >= len(stockCodeMatrix):
        break

for e in range(0, len(nameArray)):
    print(nameArray[e] + "\t" + closePriceArray[e] + "\t" + openPriceArray[e] + "\t" + differencePriceArray[e])
    #print(nameArray[e])
    #print(closePriceArray[e])
    #print(openPriceArray[e])
    #print(differencePriceArray[e])
    #print(lowestPrice[e] + "-" + highestPrice[e])
    
    
    
#for i in range(1, 50):
    #print("The " + str(i) + " one is:\n" + str(type(closePrice[i])) + "\n\n")

Completition: 0%
1
Estimated rest of the time: 15 minutes 9.983 seconds
Completition: 0%
2
Estimated rest of the time: 15 minutes 16.327 seconds
Completition: 0%
3
Estimated rest of the time: 14 minutes 29.94 seconds
Completition: 0%
4
Estimated rest of the time: 14 minutes 3.71 seconds
Completition: 0%
5
Estimated rest of the time: 14 minutes 3.986 seconds
Completition: 0%
6
Estimated rest of the time: 14 minutes 51.359 seconds
Completition: 0%
7
Estimated rest of the time: 14 minutes 38.083 seconds
Completition: 0%
8
Estimated rest of the time: 14 minutes 53.545 seconds
Completition: 0%
9
Estimated rest of the time: 14 minutes 5.732 seconds
Completition: 0%
10
Estimated rest of the time: 14 minutes 27.809 seconds
Completition: 0%
Completition: 0%
11
Estimated rest of the time: 15 minutes 13.98 seconds
Completition: 0%
12
Estimated rest of the time: 16 minutes 48.852 seconds
Completition: 0%
13
Estimated rest of the time: 24 minutes 10.95 seconds
Completition: 0%
14
Estimated rest of 

## Write The Data Into Excel File And Export It

In [31]:
import datetime
today = datetime.date.today()    # Get the date

# Load the CSV file
oldDF = pd.read_csv('example_data.csv')  
oldDF['Name of security'] = nameArray
oldDF['Close Price'] = closePriceArray
oldDF['Open Price'] = openPriceArray
oldDF['Price Range'] = differencePriceArray

# Export the file
newDF = oldDF[['Name of security', 'Close Price', 'Open Price', 'Price Range']]
newDF.to_excel('output_of_Security_' + str(today) + '.xlsx', index = False)

# The website I refered to: http://blog.alarmchang.com/?p=230